In [6]:
import pandas as pd
import numpy as np
from pycoingecko import CoinGeckoAPI as CGAPI
from datetime import datetime, timedelta
from pypfopt import expected_returns, risk_models, EfficientFrontier

In [7]:
coins = {
    "ada": "cardano",
    "hbar": "hedera-hashgraph",
    "min": "minswap",
    "vet": "vechain",
    "btc": "bitcoin",
    "eth": "ethereum",
}

# Define the CoinGeckoAPI wrapper class
class CoinGeckoAPI:
    def __init__(self):
        self.cg = CGAPI()

    def get_price_df(self, coin_id):
        end_date = datetime.today()
        start_date = end_date - timedelta(days=365)  # 1 year history

        data = self.cg.get_coin_market_chart_range_by_id(
            id=coin_id,
            vs_currency='usd',
            from_timestamp=start_date.timestamp(),
            to_timestamp=end_date.timestamp()
        )

        df = pd.DataFrame(data['prices'], columns=['timestamp', 'price'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df.set_index('timestamp', inplace=True)
        return df

# Instantiate the API
gecko = CoinGeckoAPI()

# Empty DataFrame to store prices
price_df = pd.DataFrame()

# Loop through each coin and fetch price history
for symbol, coin_id in coins.items():
    try:
        df = gecko.get_price_df(coin_id)
        price_df[symbol] = df['price']
    except Exception as e:
        print(f"Error fetching {coin_id}: {e}")

# Show the result
print(price_df.head())






                 ada      hbar       min       vet           btc          eth
timestamp                                                                    
2024-06-03  0.446384  0.099201  0.025877  0.033283  67740.016902  3780.711985
2024-06-04  0.456805  0.101076  0.026794  0.033645  68808.293686  3766.637650
2024-06-05  0.461416  0.102518  0.027068  0.034305  70600.011167  3814.932030
2024-06-06  0.461678  0.104484  0.026818  0.035587  71184.599431  3871.082091
2024-06-07  0.458149  0.101425  0.026543  0.035788  70759.588193  3812.701857


In [3]:
print(coins)


{'ada': 'cardano', 'hbar': 'hedera-hashgraph', 'min': 'minswap', 'vet': 'vechain', 'btc': 'bitcoin', 'eth': 'ethereum'}


In [4]:
price_df = price_df.resample('1D').last().dropna()
price_df = price_df.ffill().bfill()  # fill gaps


In [5]:
# Calculate expected returns and sample covariance
mu = expected_returns.mean_historical_return(price_df)
S = risk_models.sample_cov(price_df)

# Optimize portfolio for maximum Sharpe ratio
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()

print("Optimized Weights:")
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

Optimized Weights:
OrderedDict([('ada', 0.0), ('hbar', 0.00152), ('min', 0.0), ('vet', 0.0), ('btc', 0.99848), ('eth', 0.0)])
Expected annual return: 36.1%
Annual volatility: 41.2%
Sharpe Ratio: 0.88


(0.36095378145733553, 0.41241472009718705, 0.8752204125310451)